In [1]:
import tensorflow as tf
import numpy as np 
import matplotlib.pyplot as plt
import os

2024-11-20 04:04:25.545688: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732050265.696633    6506 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732050265.736070    6506 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-20 04:04:25.976346: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
train_dataset_temp = tf.keras.preprocessing.image_dataset_from_directory(
    './Datajadi/train',
    image_size=(256, 256),
    batch_size=16,
    shuffle=True,
    label_mode='categorical'
)

valid_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    './Datajadi/val',
    image_size=(256, 256),
    batch_size=16,
    label_mode='categorical'
)

test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    './Datajadi/test',
    image_size=(256, 256),
    batch_size=16,
    label_mode='categorical'
)

labelnya = train_dataset_temp.class_names
jumlah_label = len(labelnya)

Found 80800 files belonging to 101 classes.


I0000 00:00:1732050279.439993    6506 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5563 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060, pci bus id: 0000:08:00.0, compute capability: 8.9


Found 10100 files belonging to 101 classes.
Found 10100 files belonging to 101 classes.


In [3]:
data_augment = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.RandomBrightness(0.2),
])
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset_temp.map(lambda x,y: (data_augment(x, training=True),y), num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.prefetch(AUTOTUNE)
valid_dataset = valid_dataset.prefetch(AUTOTUNE)

In [4]:
with open("label_lister.txt",'w') as file:
    for i in labelnya:
        file.write(f"{i}\n")

In [5]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-6,
    verbose=1
)

In [6]:
Efficientnet = tf.keras.applications.EfficientNetV2S(
include_top=False,
weights='imagenet',
input_shape = (256,256,3)
)

In [7]:
for layer in Efficientnet.layers:
    layer.trainable= False
last_layer = Efficientnet.get_layer('block6o_add')
last_output = last_layer.output

In [8]:
x = tf.keras.layers.Conv2D(512, (3,3),activation ='relu')(last_output)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(512, activation='relu',kernel_regularizer=tf.keras.regularizers.L2(0.001))(x)
x = tf.keras.layers.Dropout(0.5)(x)
x_out = tf.keras.layers.Dense(jumlah_label, activation='softmax')(x)
model = tf.keras.Model(Efficientnet.input, x_out)
model.compile(optimizer=tf.keras.optimizers.AdamW(learning_rate=0.001,weight_decay=0.004),loss='categorical_crossentropy',metrics=['accuracy'])

In [9]:
model.fit(train_dataset, epochs=10, validation_data=valid_dataset, callbacks=[reduce_lr])

Epoch 1/10


I0000 00:00:1732050342.075016   16359 service.cc:148] XLA service 0x7fa874003320 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1732050342.075601   16359 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 4060, Compute Capability 8.9
2024-11-20 04:05:42.541829: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1732050343.991552   16359 cuda_dnn.cc:529] Loaded cuDNN version 90300
2024-11-20 04:05:49.706738: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_24419', 104 bytes spill stores, 104 bytes spill loads

2024-11-20 04:05:49.996814: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_24421', 484 

   3/5050 ━━━━━━━━━━━━━━━━━━━━ 2:49 34ms/step - accuracy: 0.0000e+00 - loss: 17.7313   

I0000 00:00:1732050362.920866   16359 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


5049/5050 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.2564 - loss: 3.7391

2024-11-20 04:08:51.393982: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_4243', 8 bytes spill stores, 8 bytes spill loads



5050/5050 ━━━━━━━━━━━━━━━━━━━━ 206s 34ms/step - accuracy: 0.2564 - loss: 3.7388 - val_accuracy: 0.4807 - val_loss: 2.1723 - learning_rate: 0.0010
Epoch 2/10
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 171s 34ms/step - accuracy: 0.4103 - loss: 2.5721 - val_accuracy: 0.5021 - val_loss: 2.0566 - learning_rate: 0.0010
Epoch 3/10
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 168s 33ms/step - accuracy: 0.4341 - loss: 2.4823 - val_accuracy: 0.5387 - val_loss: 1.9465 - learning_rate: 0.0010
Epoch 4/10
   5/5050 ━━━━━━━━━━━━━━━━━━━━ 2:29 30ms/step - accuracy: 0.4515 - loss: 2.7667 

2024-11-20 04:14:35.415822: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 12589568 bytes after encountering the first element of size 12589568 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size


5050/5050 ━━━━━━━━━━━━━━━━━━━━ 196s 39ms/step - accuracy: 0.4523 - loss: 2.4071 - val_accuracy: 0.5436 - val_loss: 1.9151 - learning_rate: 0.0010
Epoch 5/10
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 195s 39ms/step - accuracy: 0.4587 - loss: 2.3672 - val_accuracy: 0.5536 - val_loss: 1.8783 - learning_rate: 0.0010
Epoch 6/10
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 174s 34ms/step - accuracy: 0.4631 - loss: 2.3450 - val_accuracy: 0.5517 - val_loss: 1.8734 - learning_rate: 0.0010
Epoch 7/10
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 197s 39ms/step - accuracy: 0.4742 - loss: 2.3028 - val_accuracy: 0.5603 - val_loss: 1.8564 - learning_rate: 0.0010
Epoch 8/10
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 198s 39ms/step - accuracy: 0.4798 - loss: 2.2500 - val_accuracy: 0.5426 - val_loss: 1.8849 - learning_rate: 0.0010
Epoch 9/10
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 201s 40ms/step - accuracy: 0.4823 - loss: 2.2430 - val_accuracy: 0.5713 - val_loss: 1.8009 - learning_rate: 0.0010
Epoch 10/10
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 214s 42ms/step - accuracy:

In [10]:
model.save('Tranfer3/Categorical_TlEfnet_10_epochs.h5')

In [11]:
mulaidari = "block5a_expand_conv"
Kondisi = False
for layer in model.layers:
    if layer.name == mulaidari:
        Kondisi = True
    layer.trainable = Kondisi
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 256, 256,  │          0 │ input_layer_1[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 128, 128,  │        648 │ rescaling[0][0]   │
│                     │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 128, 128,  │         96 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 128, 128,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 128, 128,  │      5,184 │ stem_activation[… │
│ (Conv2D)            │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_bn  │ (None, 128, 128,  │         96 │ block1a_project_… │
│ (BatchNormalizatio… │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_ac… │ (None, 128, 128,  │          0 │ block1a_project_… │
│ (Activation)        │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_add (Add)   │ (None, 128, 128,  │          0 │ block1a_project_… │
│                     │ 24)               │            │ stem_activation[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_project_co… │ (None, 128, 128,  │      5,184 │ block1a_add[0][0] │
│ (Conv2D)            │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_project_bn  │ (None, 128, 128,  │         96 │ block1b_project_… │
│ (BatchNormalizatio… │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_project_ac… │ (None, 128, 128,  │          0 │ block1b_project_… │
│ (Activation)        │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_drop        │ (None, 128, 128,  │          0 │ block1b_project_… │
│ (Dropout)           │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_add (Add)   │ (None, 128, 128,  │          0 │ block1b_drop[0][… │
│                     │ 24)               │            │ block1a_add[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_expand_conv │ (None, 64, 64,    │     20,736 │ block1b_add[0][0] │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_expand_bn   │ (None, 64, 64,    │        384 │ block2a_expand_c… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_expand_act… │ (None, 64, 64,    │          0 │ block2a_expand_b

 Total params: 24,482,449 (93.39 MB)

 Trainable params: 19,520,301 (74.46 MB)

 Non-trainable params: 1,972,888 (7.53 MB)

 Optimizer params: 2,989,260 (11.40 MB)

In [12]:
model.fit(train_dataset, epochs=20, validation_data=valid_dataset, callbacks=[reduce_lr],initial_epoch=10)

Epoch 11/20
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 228s 45ms/step - accuracy: 0.4933 - loss: 2.1919 - val_accuracy: 0.5686 - val_loss: 1.8255 - learning_rate: 0.0010
Epoch 12/20
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 247s 49ms/step - accuracy: 0.4947 - loss: 2.1821 - val_accuracy: 0.5628 - val_loss: 1.8443 - learning_rate: 0.0010
Epoch 13/20
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 231s 46ms/step - accuracy: 0.4991 - loss: 2.1465 - val_accuracy: 0.5744 - val_loss: 1.8019 - learning_rate: 0.0010
Epoch 14/20
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 202s 40ms/step - accuracy: 0.4984 - loss: 2.1461 - val_accuracy: 0.5659 - val_loss: 1.8153 - learning_rate: 0.0010
Epoch 15/20
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 195s 39ms/step - accuracy: 0.5041 - loss: 2.1196 - val_accuracy: 0.5818 - val_loss: 1.7771 - learning_rate: 0.0010
Epoch 16/20
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 204s 40ms/step - accuracy: 0.5058 - loss: 2.1130 - val_accuracy: 0.5844 - val_loss: 1.7795 - learning_rate: 0.0010
Epoch 17/20
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 196s 39ms

In [13]:
model.save('Tranfer3/Categorical_TlEfnet_20_epochs.h5')

In [14]:
val_loss, val_acc = model.evaluate(valid_dataset)
tra_loss, tra_acc = model.evaluate(train_dataset)
test_loss, test_acc = model.evaluate(test_dataset)


print(f"Train Accuracy\t\t: {tra_acc * 100:.2f}% | Train Loss\t\t: {tra_loss}")
print(f"Validation Accuracy\t: {val_acc * 100:.2f}% | Validation Loss\t: {val_loss}")
print(f"Test Accuracy\t\t: {test_acc * 100:.2f}% | Test Loss \t\t: {test_loss}")

632/632 ━━━━━━━━━━━━━━━━━━━━ 18s 28ms/step - accuracy: 0.5729 - loss: 1.7779
   5/5050 ━━━━━━━━━━━━━━━━━━━━ 2:31 30ms/step - accuracy: 0.5700 - loss: 1.6064 

2024-11-20 05:12:42.278935: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 12589568 bytes after encountering the first element of size 12589568 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size


5050/5050 ━━━━━━━━━━━━━━━━━━━━ 147s 29ms/step - accuracy: 0.5878 - loss: 1.7007
632/632 ━━━━━━━━━━━━━━━━━━━━ 14s 23ms/step - accuracy: 0.5742 - loss: 1.7379
Train Accuracy		: 58.95% | Train Loss		: 1.6715033054351807
Validation Accuracy	: 58.12% | Validation Loss	: 1.7600653171539307
Test Accuracy		: 57.74% | Test Loss 		: 1.7684179544448853


In [15]:
model.fit(train_dataset, epochs=60, validation_data=valid_dataset, callbacks=[reduce_lr],initial_epoch=20)

Epoch 21/60
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 192s 38ms/step - accuracy: 0.5434 - loss: 1.9142 - val_accuracy: 0.6228 - val_loss: 1.5799 - learning_rate: 5.0000e-04
Epoch 22/60
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 196s 39ms/step - accuracy: 0.5628 - loss: 1.8075 - val_accuracy: 0.6210 - val_loss: 1.5670 - learning_rate: 5.0000e-04
Epoch 23/60
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 191s 38ms/step - accuracy: 0.5659 - loss: 1.7822 - val_accuracy: 0.6248 - val_loss: 1.5368 - learning_rate: 5.0000e-04
Epoch 24/60
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 164s 32ms/step - accuracy: 0.5691 - loss: 1.7683 - val_accuracy: 0.6284 - val_loss: 1.5244 - learning_rate: 5.0000e-04
Epoch 25/60
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 164s 32ms/step - accuracy: 0.5739 - loss: 1.7444 - val_accuracy: 0.6180 - val_loss: 1.5299 - learning_rate: 5.0000e-04
Epoch 26/60
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 167s 33ms/step - accuracy: 0.5743 - loss: 1.7450 - val_accuracy: 0.6283 - val_loss: 1.5157 - learning_rate: 5.0000e-04
Epoch 27/60
5050/5050 ━━━━━━

2024-11-20 05:40:14.040500: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 12589568 bytes after encountering the first element of size 12589568 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size


5050/5050 ━━━━━━━━━━━━━━━━━━━━ 180s 36ms/step - accuracy: 0.5783 - loss: 1.7206 - val_accuracy: 0.6301 - val_loss: 1.5017 - learning_rate: 5.0000e-04
Epoch 30/60
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 190s 38ms/step - accuracy: 0.5743 - loss: 1.7267 - val_accuracy: 0.6305 - val_loss: 1.5015 - learning_rate: 5.0000e-04
Epoch 31/60
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 198s 39ms/step - accuracy: 0.5756 - loss: 1.7236 - val_accuracy: 0.6296 - val_loss: 1.5064 - learning_rate: 5.0000e-04
Epoch 32/60
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 193s 38ms/step - accuracy: 0.5785 - loss: 1.7047 - val_accuracy: 0.6334 - val_loss: 1.4885 - learning_rate: 5.0000e-04
Epoch 33/60
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 197s 39ms/step - accuracy: 0.5833 - loss: 1.7014 - val_accuracy: 0.6308 - val_loss: 1.5050 - learning_rate: 5.0000e-04
Epoch 34/60
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 195s 39ms/step - accuracy: 0.5853 - loss: 1.6968 - val_accuracy: 0.6350 - val_loss: 1.4831 - learning_rate: 5.0000e-04
Epoch 35/60
5050/5050 ━━━━━━━━━━━━━━━━━━

2024-11-20 06:13:49.890147: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 12589568 bytes after encountering the first element of size 12589568 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size


5050/5050 ━━━━━━━━━━━━━━━━━━━━ 204s 40ms/step - accuracy: 0.5891 - loss: 1.6656 - val_accuracy: 0.6376 - val_loss: 1.4839 - learning_rate: 5.0000e-04
Epoch 40/60
5049/5050 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.5870 - loss: 1.6724
Epoch 40: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 220s 43ms/step - accuracy: 0.5870 - loss: 1.6724 - val_accuracy: 0.6342 - val_loss: 1.4974 - learning_rate: 5.0000e-04
Epoch 41/60
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 196s 39ms/step - accuracy: 0.6015 - loss: 1.6048 - val_accuracy: 0.6574 - val_loss: 1.4076 - learning_rate: 2.5000e-04
Epoch 42/60
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 199s 39ms/step - accuracy: 0.6093 - loss: 1.5541 - val_accuracy: 0.6536 - val_loss: 1.3953 - learning_rate: 2.5000e-04
Epoch 43/60
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 199s 39ms/step - accuracy: 0.6130 - loss: 1.5333 - val_accuracy: 0.6538 - val_loss: 1.3883 - learning_rate: 2.5000e-04
Epoch 44/60
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 195s

...

5050/5050 ━━━━━━━━━━━━━━━━━━━━ 186s 37ms/step - accuracy: 0.6216 - loss: 1.4963 - val_accuracy: 0.6595 - val_loss: 1.3752 - learning_rate: 2.5000e-04
Epoch 51/60
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 205s 41ms/step - accuracy: 0.6221 - loss: 1.4858 - val_accuracy: 0.6633 - val_loss: 1.3512 - learning_rate: 2.5000e-04
Epoch 52/60
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 207s 41ms/step - accuracy: 0.6249 - loss: 1.4777 - val_accuracy: 0.6650 - val_loss: 1.3586 - learning_rate: 2.5000e-04
Epoch 53/60
   3/5050 ━━━━━━━━━━━━━━━━━━━━ 3:09 38ms/step - accuracy: 0.8160 - loss: 1.1152  

2024-11-20 07:02:40.280739: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 12589568 bytes after encountering the first element of size 12589568 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size


5050/5050 ━━━━━━━━━━━━━━━━━━━━ 209s 41ms/step - accuracy: 0.6269 - loss: 1.4845 - val_accuracy: 0.6584 - val_loss: 1.3701 - learning_rate: 2.5000e-04
Epoch 54/60
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.6282 - loss: 1.4709
Epoch 54: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 229s 45ms/step - accuracy: 0.6282 - loss: 1.4709 - val_accuracy: 0.6638 - val_loss: 1.3592 - learning_rate: 2.5000e-04
Epoch 55/60
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 218s 43ms/step - accuracy: 0.6341 - loss: 1.4299 - val_accuracy: 0.6714 - val_loss: 1.3280 - learning_rate: 1.2500e-04
Epoch 56/60
   3/5050 ━━━━━━━━━━━━━━━━━━━━ 2:59 35ms/step - accuracy: 0.6319 - loss: 1.3688  

2024-11-20 07:13:36.027668: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 12589568 bytes after encountering the first element of size 12589568 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size


5050/5050 ━━━━━━━━━━━━━━━━━━━━ 217s 43ms/step - accuracy: 0.6396 - loss: 1.4165 - val_accuracy: 0.6727 - val_loss: 1.3259 - learning_rate: 1.2500e-04
Epoch 57/60
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 209s 41ms/step - accuracy: 0.6421 - loss: 1.4122 - val_accuracy: 0.6747 - val_loss: 1.3192 - learning_rate: 1.2500e-04
Epoch 58/60
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 236s 47ms/step - accuracy: 0.6439 - loss: 1.4068 - val_accuracy: 0.6750 - val_loss: 1.3229 - learning_rate: 1.2500e-04
Epoch 59/60
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 251s 50ms/step - accuracy: 0.6430 - loss: 1.3949 - val_accuracy: 0.6785 - val_loss: 1.3091 - learning_rate: 1.2500e-04
Epoch 60/60
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 227s 45ms/step - accuracy: 0.6458 - loss: 1.3871 - val_accuracy: 0.6794 - val_loss: 1.3029 - learning_rate: 1.2500e-04


In [16]:
model.save('Tranfer4/Categorical_TlEfnet_60_epochs.h5')

In [17]:
val_loss, val_acc = model.evaluate(valid_dataset)
tra_loss, tra_acc = model.evaluate(train_dataset)
test_loss, test_acc = model.evaluate(test_dataset)


print(f"Train Accuracy\t\t: {tra_acc * 100:.2f}% | Train Loss\t\t: {tra_loss}")
print(f"Validation Accuracy\t: {val_acc * 100:.2f}% | Validation Loss\t: {val_loss}")
print(f"Test Accuracy\t\t: {test_acc * 100:.2f}% | Test Loss \t\t: {test_loss}")

632/632 ━━━━━━━━━━━━━━━━━━━━ 19s 30ms/step - accuracy: 0.6764 - loss: 1.3115
   2/5050 ━━━━━━━━━━━━━━━━━━━━ 10:04 120ms/step - accuracy: 0.7188 - loss: 1.0410

2024-11-20 07:32:57.211203: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 12589568 bytes after encountering the first element of size 12589568 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size


5050/5050 ━━━━━━━━━━━━━━━━━━━━ 144s 28ms/step - accuracy: 0.7105 - loss: 1.0787
632/632 ━━━━━━━━━━━━━━━━━━━━ 15s 24ms/step - accuracy: 0.6716 - loss: 1.2815
Train Accuracy		: 71.22% | Train Loss		: 1.069667935371399
Validation Accuracy	: 67.94% | Validation Loss	: 1.3028984069824219
Test Accuracy		: 67.12% | Test Loss 		: 1.3145737648010254


In [18]:
mulaidari = "block3a_expand_conv"
Kondisi = False
for layer in model.layers:
    if layer.name == mulaidari:
        Kondisi = True
    layer.trainable = Kondisi
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 256, 256,  │          0 │ input_layer_1[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 128, 128,  │        648 │ rescaling[0][0]   │
│                     │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 128, 128,  │         96 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 128, 128,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 128, 128,  │      5,184 │ stem_activation[… │
│ (Conv2D)            │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_bn  │ (None, 128, 128,  │         96 │ block1a_project_… │
│ (BatchNormalizatio… │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_ac… │ (None, 128, 128,  │          0 │ block1a_project_… │
│ (Activation)        │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_add (Add)   │ (None, 128, 128,  │          0 │ block1a_project_… │
│                     │ 24)               │            │ stem_activation[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_project_co… │ (None, 128, 128,  │      5,184 │ block1a_add[0][0] │
│ (Conv2D)            │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_project_bn  │ (None, 128, 128,  │         96 │ block1b_project_… │
│ (BatchNormalizatio… │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_project_ac… │ (None, 128, 128,  │          0 │ block1b_project_… │
│ (Activation)        │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_drop        │ (None, 128, 128,  │          0 │ block1b_project_… │
│ (Dropout)           │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_add (Add)   │ (None, 128, 128,  │          0 │ block1b_drop[0][… │
│                     │ 24)               │            │ block1a_add[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_expand_conv │ (None, 64, 64,    │     20,736 │ block1b_add[0][0] │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_expand_bn   │ (None, 64, 64,    │        384 │ block2a_expand_c… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_expand_act… │ (None, 64, 64,    │          0 │ block2a_expand_b

 Total params: 24,482,449 (93.39 MB)

 Trainable params: 21,027,165 (80.21 MB)

 Non-trainable params: 466,024 (1.78 MB)

 Optimizer params: 2,989,260 (11.40 MB)

In [19]:
model.fit(train_dataset, epochs=80, validation_data=valid_dataset, callbacks=[reduce_lr], initial_epoch=60)

Epoch 61/80
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 159s 31ms/step - accuracy: 0.6440 - loss: 1.3886 - val_accuracy: 0.6768 - val_loss: 1.3059 - learning_rate: 1.2500e-04
Epoch 62/80
   5/5050 ━━━━━━━━━━━━━━━━━━━━ 2:57 35ms/step - accuracy: 0.6515 - loss: 1.3739 

2024-11-20 07:55:41.007108: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 12589568 bytes after encountering the first element of size 12589568 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size


5050/5050 ━━━━━━━━━━━━━━━━━━━━ 276s 55ms/step - accuracy: 0.6445 - loss: 1.3920 - val_accuracy: 0.6793 - val_loss: 1.2935 - learning_rate: 1.2500e-04
Epoch 63/80
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 206s 41ms/step - accuracy: 0.6462 - loss: 1.3827 - val_accuracy: 0.6774 - val_loss: 1.3046 - learning_rate: 1.2500e-04
Epoch 64/80
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 199s 39ms/step - accuracy: 0.6473 - loss: 1.3750 - val_accuracy: 0.6777 - val_loss: 1.2949 - learning_rate: 1.2500e-04
Epoch 65/80
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 207s 41ms/step - accuracy: 0.6494 - loss: 1.3703 - val_accuracy: 0.6790 - val_loss: 1.2900 - learning_rate: 1.2500e-04
Epoch 66/80
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 207s 41ms/step - accuracy: 0.6495 - loss: 1.3677 - val_accuracy: 0.6852 - val_loss: 1.2865 - learning_rate: 1.2500e-04
Epoch 67/80
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 234s 46ms/step - accuracy: 0.6525 - loss: 1.3550 - val_accuracy: 0.6787 - val_loss: 1.2901 - learning_rate: 1.2500e-04
Epoch 68/80
5050/5050 ━━━━━━━━━━━━━━━━━━

In [20]:
model.save('Tranfer4/Categorical_Tlnet_80_epochs.h5')

In [21]:
model.fit(train_dataset, epochs=100, validation_data=valid_dataset, callbacks=[reduce_lr], initial_epoch=80)

Epoch 81/100
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 207s 41ms/step - accuracy: 0.6656 - loss: 1.2856 - val_accuracy: 0.6933 - val_loss: 1.2435 - learning_rate: 6.2500e-05
Epoch 82/100
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 205s 41ms/step - accuracy: 0.6699 - loss: 1.2758 - val_accuracy: 0.6922 - val_loss: 1.2418 - learning_rate: 6.2500e-05
Epoch 83/100
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 208s 41ms/step - accuracy: 0.6676 - loss: 1.2843 - val_accuracy: 0.6924 - val_loss: 1.2448 - learning_rate: 6.2500e-05
Epoch 84/100
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 204s 40ms/step - accuracy: 0.6719 - loss: 1.2796 - val_accuracy: 0.6910 - val_loss: 1.2468 - learning_rate: 6.2500e-05
Epoch 85/100
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 210s 41ms/step - accuracy: 0.6677 - loss: 1.2800 - val_accuracy: 0.6905 - val_loss: 1.2343 - learning_rate: 6.2500e-05
Epoch 86/100
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 205s 41ms/step - accuracy: 0.6685 - loss: 1.2788 - val_accuracy: 0.6923 - val_loss: 1.2393 - learning_rate: 6.2500e-05
Epoch 87/100
   5/5050

2024-11-20 09:26:38.162954: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 12589568 bytes after encountering the first element of size 12589568 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size


5050/5050 ━━━━━━━━━━━━━━━━━━━━ 275s 54ms/step - accuracy: 0.6727 - loss: 1.2697 - val_accuracy: 0.6942 - val_loss: 1.2371 - learning_rate: 6.2500e-05
Epoch 88/100
5049/5050 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.6725 - loss: 1.2653
Epoch 88: ReduceLROnPlateau reducing learning rate to 3.125000148429535e-05.
5050/5050 ━━━━━━━━━━━━━━━━━━━━ 214s 42ms/step - accuracy: 0.6725 - loss: 1.2653 - val_accuracy: 0.6938 - val_loss: 1.2406 - learning_rate: 6.2500e-05
Epoch 89/100
   6/5050 ━━━━━━━━━━━━━━━━━━━━ 2:09 26ms/step - accuracy: 0.6562 - loss: 1.4555 

2024-11-20 09:34:47.074130: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 12589568 bytes after encountering the first element of size 12589568 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size


5050/5050 ━━━━━━━━━━━━━━━━━━━━ 198s 39ms/step - accuracy: 0.6753 - loss: 1.2607 - val_accuracy: 0.6952 - val_loss: 1.2274 - learning_rate: 3.1250e-05
Epoch 90/100
 621/5050 ━━━━━━━━━━━━━━━━━━━━ 2:58 40ms/step - accuracy: 0.6671 - loss: 1.2841

KeyboardInterrupt: 

In [22]:
model.save('Tranfer4/Categorical_Tlnet_89_epochs.keras')